<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf sample_data

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


## **Installation de librairies**

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import os
import numpy as np
np.set_printoptions(edgeitems = 3,
                    infstr = 'inf',
                    linewidth = 150, 
                    nanstr = 'nan', precision = 3, 
                    suppress = False, 
                    threshold = 1000, 
                    formatter = None)

# **1. Chargement des bases de données**

## 1.1. Sklearn sur le jeu de données 20newsgroup

Commençons par importer le jeu de données 20newsgroup (https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). Ce jeu de données contient 18 000 news sur 20 thématiques différentes. Il est également divisé en deux sous-ensembles: le *training set*, pour entrainer un modèle, et le *test set* pour évaluer le modèle entrainé.

Dans le bloc de code suivant, nous importons le *training set* (subset='train'). Nous précisons également que les données de cet ensemble doivent être mélangées (shuffle=True). La variable *random_state* permet de fixer la *seed* des opérations aléatoires, ce qui garantit d'obtenir le même résultat à chaque exécution.

In [4]:
from sklearn.datasets import fetch_20newsgroups

news_ds_train = fetch_20newsgroups(subset = 'train', shuffle = True, random_state = 42)

Dans le bloc de code suivant, plusieurs éléments sont affichés :
* La liste des topics abordés dans le jeu de données
* Le nombre de news 
* Le contenu de la 1ère news
* Le topic de la 1ère news

In [8]:
print("Liste des topics")
print(news_ds_train.target_names)
print("\nNombre de news")
print(len(news_ds_train.data))
print("\nAffichage de la 1ère news")
print(news_ds_train.data[0])
print("\nTopic de la 1ère news")
print(news_ds_train.target_names[news_ds_train.target[0]])

Liste des topics
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

Nombre de news
11314

Affichage de la 1ère news
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of produc

Si nous ne souhaitons pas travailler avec tous les topics, il est possible de déterminer dès le chargement du jeu de données les catégories que l'on souhaite utiliser.

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
news_ds_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
print(news_ds_train.target_names)

## 1.2. Huggingface sur le jeu de données 20newsgroup

Le chargement du jeu de données avec Huggingface est très similaire à celui de sklearn, à quelques détails près. Le bloc de code suivant vous montre comment procéder avec Huggingface pour obtenir les mêmes résultats qu'avec sklearn.

In [ ]:
from datasets import load_dataset

hugging_news_train = load_dataset('SetFit/20_newsgroups', split = 'train')
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
hugging_news_train = hugging_news_train.filter(lambda row: row['label_text'] in categories)

print("Nombre de news")
print(len(hugging_news_train))
print("\nAffichage de la 1ère news")
print(hugging_news_train[0])
print("\nTopic de la 1ère news")
print(hugging_news_train[0]['label_text'])

# **2. Classification de textes**

## 2.1. Classification avec sklearn
Dans cette section, nous allons entrainer un modèle pour classifier les news du jeu de données 20newsgroup avec la librairie sklearn.

Dans le bloc de code suivant, commencez par importer le jeu de données comme nous l'avons vu au point 1.1. Conservez les topics suivants : 
* alt.atheism
* comp.graphics
* rec.motorcycles
* sci.crypt
* sci.med

Ne mélangez pas les news du *test set*.

Sur base de ce que nous avons fait la semaine passée, comptez le nombre d'occurence des mots dans le corpus que nous venons de créer (ce corpus correspond à la variable *news_ds_train.data*).

Vectorisez ensuite ce corpus avec TF-IDF. Enregistrez ces features du corpus dans la variable *corpus_X*.

In [28]:
categories = ['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt','sci.med']
news_ds_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)
print(news_ds_train.target_names)

categories = ['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt','sci.med']
news_ds_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)

['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt', 'sci.med']


In [34]:
import sklearn.feature_extraction.text as skl_text

vectorizer = skl_text.TfidfVectorizer()
corpus_X = vectorizer.fit_transform(news_ds_train.data).toarray()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(corpus_X, news_ds_train.target, test_size = 0.25, random_state = 0)

print(vectorizer.get_feature_names_out())
print(len(vectorizer.get_feature_names_out()))

['00' '000' '0000' ... 'érale' 'ête' 'íålittin']
42672


Nous allons maintenant entrainer un modèle appelé *Multinomial Naive Bayes* sur ce corpus. Les labels du corpus se trouvent dans la variable *news_ds_train.target*.

In [35]:
from sklearn.naive_bayes import MultinomialNB
clf_sklearn = MultinomialNB().fit(X_train, y_train)

Dans le bloc de code suivant, nous utilisons notre modèle entrainé sur des phrases pré-sélectionnées. Est-ce que votre modèle semble performant par rapport aux thématiques choisies ?

In [27]:
docs_new = ['Your post is based on the premise that the laws as they stand do not discriminate anybody', 'OpenGL on the GPU is fast']

# Vectorisez avec TF-IDF ces phrases
X_new_tfidf = vectorizer.fit_transform(docs_new)

predicted = clf_sklearn.predict(corpus_X)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, news_ds_train.target_names[category]))

'Your post is based on the premise that the laws as they stand do not discriminate anybody' => sci.med
'OpenGL on the GPU is fast' => comp.graphics


## 2.2. Classification avec les transformers de Huggingface

Dans cette section, nous vous proposons d'entrainer un modèle pour classifier les news du jeu de données 20newsgroup avec la librairie transformer de Huggingface.

Nous vous suggérons d'explorer l'utilisation de cette librairie sur base du lien suivant : https://huggingface.co/docs/transformers/tasks/sequence_classification 

In [ ]:
# Votre code ici

# 3. Métriques de classification

## 3.1. Métriques de la classification avec sklearn

Dans la section 2.1, nous avons entrainé un modèle de classification avec sklearn. Nous allons maintenant voir comment mesurer ses performances.

Dans le bloc de code suivant, nous utilisons notre modèle sur le corpus d'entrainement afin de déterminer ses prédictions pour chaque news d'entrainement. Nous utilisons ensuite la fonction *classification_report* pour mesurer différentes métriques sur le modèle.

In [36]:
predicted = clf_sklearn.predict(corpus_X)

from sklearn import metrics
print(metrics.classification_report(news_ds_train.target, predicted, target_names = news_ds_train.target_names))

                 precision    recall  f1-score   support

    alt.atheism       1.00      0.99      1.00       480
  comp.graphics       0.99      0.96      0.97       584
rec.motorcycles       1.00      0.99      0.99       598
      sci.crypt       0.94      0.99      0.97       595
        sci.med       0.99      0.98      0.99       594

       accuracy                           0.98      2851
      macro avg       0.98      0.98      0.98      2851
   weighted avg       0.98      0.98      0.98      2851



En pratique, pour comparer des modèles entrainés, nous les évaluons sur le *test set*. Evaluez le modèle entrainé avec sklearn sur notre test set.

In [39]:
predicted = clf_sklearn.predict(X_test)

from sklearn import metrics
print(metrics.classification_report(y_test, predicted, target_names = news_ds_test.target_names))

                 precision    recall  f1-score   support

    alt.atheism       1.00      0.99      1.00       121
  comp.graphics       0.97      0.90      0.93       143
rec.motorcycles       0.99      0.97      0.98       159
      sci.crypt       0.88      0.99      0.93       143
        sci.med       0.99      0.97      0.98       147

       accuracy                           0.96       713
      macro avg       0.97      0.96      0.96       713
   weighted avg       0.96      0.96      0.96       713



Essayez maintenant d'entrainer un modèle MLPClassifier de sklearn. Aidez-vous de la procédure complète de l'entrainement effectué pour sklearn et comparez votre modèle à celui entrainé précédement.

In [ ]:
from sklearn.neural_network import MLPClassifier

# Chargez les datasets en conservant les mêmes catégories
# Votre code ici

# Utilisez TF-IDF pour convertir les données
# Votre code ici

# Initializez le MLPClassifier et entrainez-le
# Votre code ici

# Evaluez le modèle sur le training set
# Votre code ici

# Evaluez le modèle sur le test set
# Votre code ici

# 4. Modèles de langage

Dans cette section, nous allons travailler avec les modèles de langage et voir différentes tâches sur lesquelles ils peuvent être utilisées.

In [40]:
# Ce bloc charge des librairies qui seront utilisées dans la suite de cette section
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from tqdm import tqdm

## 4.1. Prédire la suite d'une phrase

Un modèle de language peut être utilisé pour prédire le prochain mot d'une phrase. Pour ce faire nous utiliserons l'identifiant 'distilgpt2' et des modèles basés sur GPT-2 (voir bloc de code ci-dessous).

In [ ]:
model_id = 'distilgpt2'
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

Le bloc de code suivant vous montre comment utiliser ces modèles pour générer la suite d'une phrase. Essayez différentes phrases et différents nombres de mots à générer. 

En cherchant en ligne les différentes fonctions utilisées dans ce bout de code, essayez de comprendre ce qui y est fait.

In [46]:
sentence = "The human reads a"
encodings = tokenizer(sentence, return_tensors='pt')
print(encodings)

words_to_generate = 5 # Nombre de mots à générer à la suite de la phrase d'exemple

for i in tqdm(range(words_to_generate)):
  with torch.no_grad():
    encodings.input_ids = encodings.input_ids
    output = model(encodings.input_ids).logits[:, -1, :]
    # filter
    filtered_next_token_logits = top_k_top_p_filtering(output, top_k=50, top_p=1.0)

    # sample
    probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    encodings.input_ids = torch.cat((encodings.input_ids, next_token), dim=-1)
    encodings.attention_mask = torch.cat((encodings.attention_mask, torch.tensor([[1]])), dim=-1)


resulting_string = tokenizer.decode(encodings.input_ids.tolist()[0])
print()
print(resulting_string)

{'input_ids': tensor([[ 464, 1692, 9743,  257]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


100%|██████████| 5/5 [00:00<00:00,  8.97it/s]


The human reads a paragraph or two or more


## 4.2. Fill-mask

Dans cette tâche, un mot de la phrase est manquante (marquée par le token [MASK]). Un modèle de langage peut être utilisé pour calculer le mot le plus probable de la phrase.

In [47]:
from transformers import pipeline

lm_unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
lm_unmasker("The human reads a [MASK]")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.32789847254753113,
  'token': 1012,
  'token_str': '.',
  'sequence': 'the human reads a.'},
 {'score': 0.06196603178977966,
  'token': 5961,
  'token_str': 'poem',
  'sequence': 'the human reads a poem'},
 {'score': 0.033230192959308624,
  'token': 1024,
  'token_str': ':',
  'sequence': 'the human reads a :'},
 {'score': 0.03241956606507301,
  'token': 999,
  'token_str': '!',
  'sequence': 'the human reads a!'},
 {'score': 0.02792378142476082,
  'token': 3661,
  'token_str': 'letter',
  'sequence': 'the human reads a letter'}]

Le mot manquant ne doit pas toujours être situé en fin de phrase :

In [48]:
lm_unmasker("Paris is the [MASK] of France.")

[{'score': 0.9815465807914734,
  'token': 3007,
  'token_str': 'capital',
  'sequence': 'paris is the capital of france.'},
 {'score': 0.0033424370922148228,
  'token': 14508,
  'token_str': 'birthplace',
  'sequence': 'paris is the birthplace of france.'},
 {'score': 0.0010447038803249598,
  'token': 22037,
  'token_str': 'northernmost',
  'sequence': 'paris is the northernmost of france.'},
 {'score': 0.001004314050078392,
  'token': 2803,
  'token_str': 'centre',
  'sequence': 'paris is the centre of france.'},
 {'score': 0.0007803135085850954,
  'token': 21787,
  'token_str': 'southernmost',
  'sequence': 'paris is the southernmost of france.'}]

## 4.3. Perplexité

La perplexité d'un modèle du langage est inversément proportionnelle à la qualité de ce modèle. Une valeur de 1 (au plus petit, au mieux c'est) serait obtenue avec un modèle parfait. Une perplexité de 30 (à titre d'exemple) serait obtenu avec un modèle plutôt hésitant.

Le bout de code charge une portion d'une variante du jeu de données 20 newsgroup. Le code qui suit vous montre comment calculer la perplexité de notre modèle du langage.

In [ ]:
test = load_dataset('newsgroup', 'bydate_alt.atheism', split='test')
print('\n\n'.join(test['text']))
encodings = tokenizer('\n\n'.join(test['text'][:10]), return_tensors='pt')
print(encodings.input_ids.shape)

In [51]:
context_length = 256
stride = 512

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - context_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    if end_loc <= begin_loc:
      break

    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
print()
print(ppl)

 93%|█████████▎| 13/14 [00:09<00:00,  1.34it/s]


tensor(49.3254)


# 5. Data Augmentation

## 5.1. Augmentation de données avec word2vec

Le bout de code suivant prépare le modèle word2vec pour vous.

In [ ]:
import os
import gensim
from gensim import downloader
# Model loading
if os.path.isdir("gensim-data"):
  from gensim.models import KeyedVectors
  glove_model_en = KeyedVectors.load_word2vec_format(os.path.join('gensim-data', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-50.gz'))
else:
  glove_model_en = downloader.load("glove-wiki-gigaword-50")
  os.system('cp -R /root/gensim-data ./gensim-data')

print("Loaded vocab size %i" % len(glove_model_en.vocab.keys()))

model = glove_model_en

Le script suivant permet de remplacer les mots d'une phrase par les mots les plus similaires à ceux-ci. Observez le résultat. Quels sont les problèmes que l'on rencontre ?

In [ ]:
sentence = "The boy is running on the field"
augmented_sentence = []
for w in sentence.lower().split(" "):
  augmented_sentence.append(model.most_similar(w)[0][0])

augmented_sentence = ' '.join(augmented_sentence)
print(augmented_sentence)

Une solution à ces problèmes est de ne remplacer que certains mots (les noms ou les verbes). Utilisez la librairie spacy (cfr. séance précédente) pour remplacer chaque nom de la phrase par son mot le plus similaire.

Par facilité, nous ne tiendrons pas compte des ponctuations.

In [ ]:
# votre code ici

Faites-en de même, mais en remplaçant les verbes !

In [ ]:
# votre code ici

Quelles obsevations peut-on faire ?

*Votre réponse ici*

## 5.2. Augmentation de données avec WordNet

WordNet permet de trouver les mots synonymes d'un autre mot. Dans le code suivant, nous vous proposons une fonction utilisant WordNet pour trouver une liste de synonymes pour un mot donné.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import wordnet

from collections import OrderedDict
from nltk.tokenize import word_tokenize
def find_synonyms(word):
  synonyms = []
  for synset in wordnet.synsets(word):
    for syn in synset.lemma_names():
      synonyms.append(syn)

  # using this to drop duplicates while maintaining word order (closest synonyms comes first)
  synonyms_without_duplicates = list(OrderedDict.fromkeys(synonyms))
  synonyms_without_duplicates.remove(word) # remove the word if it's in the list
  return synonyms_without_duplicates

Nous pouvons vérifier cette fonction avec le code suivant

In [ ]:
synonyms = find_synonyms("boy")
print(synonyms)

Proposez maintenant un script similaire à ceux utilisés dans la Section 5.1 en utilisant WordNet pour remplacer les noms de la phrase par un synonyme.

In [ ]:
# votre code ici

Quelles obsevations peut-on faire ?

*Votre réponse ici*

## 5.3. Augmentation de données avec un modèle de language

En remplaçant les noms d'une phrase par le token [MASK], nous pouvons utiliser la tâche fill-mask et un modèle du langage (section 4.2) pour trouver des idées pour créer de nouvelles phrases.

Implémentez cette idée dans le bloc de code suivant.

In [ ]:
# votre code ici

Quelles obsevations peut-on faire ?

*Votre réponse ici*

## 5.4. Autres méthodes d'augmentation de données pour le NLP

D'autres approches sont possibles pour augmenter les données dans une tâche de NLP. Nous vous invitons à consulter les documents suivants et de tenter d'expérimenter par vous-mêmes.

* https://neptune.ai/blog/data-augmentation-nlp
* https://github.com/makcedward/nlpaug

In [ ]:
# vos expérimentations ici